
#Installation
---



In [1]:
# !pip install pyarrow

In [2]:
from pathlib import Path
import os
from datetime import date
import re

In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import box
import dask.dataframe as dd
pd.options.mode.chained_assignment = None

In [4]:
# Paths
p = Path('Z:\FFE')
pathShapefile = Path(p / 'shapefile')
pathParquets = Path(p / 'output')

print(len(os.listdir(pathParquets)))

3000


In [5]:
def get_list_pids(path=pathParquets):
    pidList = []
    updtScenariolist = []
    files = pathParquets.glob('*.parquet')
    for file in files:
        regex = r"pid\d*"
        pidNames = re.findall(regex, str(file))
        for pidName in pidNames:
          print(f" file pid {pidName}")
          pidList.append(pidName)
    pidList = list(set(pidList))
    updtScenariolist = np.arange(0, 3000, 300).tolist()
    return pidList, updtScenariolist


def read_and_concatenate_parquets(pidList,incrList,path=pathParquets):
    L = []
    for idx, val in enumerate(pidList):
        files = pathParquets.glob('*'+str(val)+'.parquet')
        for file in files:
            print(file)
            pqt = pd.read_parquet(file, engine='pyarrow')
            pqt['scenario']=pqt['scenario']+incrList[idx]
#             print(pqt.head())
            L.append(pqt)
            print(len(L))
    df = dd.concat(L)
    return df

    
def count_fid_occurences(df):
    df.drop_duplicates(subset=['scenario', 'source'], inplace = True) 
    count_df = df['source'].value_counts().compute()
    count_df = pd.DataFrame({'source': count_df.index, 'count': count_df.values})
    count_df.to_parquet(str(pathShapefile) + '/' + f'CountBurn-{str(date.today())}.parquet', 
                        engine='auto', compression="GZIP")# could be datetime.now
    return count_df


def Merge(countDf,nameShapefile):
  # Shapefile
    gdfShape = gpd.read_file(pathShapefile / nameShapefile)
    gdfShape.insert(0, 'FID', range(0, len(gdfShape)))
    gdfShape.rename(columns={'FID': 'source'}, inplace=True)
    gdfShape = gdfShape[['source', 'geometry']]
    countDf.index.name = None
    merged = countDf.merge(gdfShape, on=['source'], how='left')
    return merged


def createShapefile(df):
  # df = df.to_pandas()
  gdf = gpd.GeoDataFrame(df, geometry='geometry')
  # gdf.plot(column='count', cmap='hsv', legend=True)
  gdf.to_file(os.path.join(str(pathShapefile) + "/" + "Burn3000scenarioWellington.shp"))
  return gdf

In [6]:
pids, incr = get_list_pids()

 file pid pid66556
 file pid pid66557
 file pid pid66558
 file pid pid66559
 file pid pid66560
 file pid pid73177
 file pid pid73178
 file pid pid73179
 file pid pid73180
 file pid pid73181
 file pid pid66556
 file pid pid66557
 file pid pid66558
 file pid pid66559
 file pid pid66560
 file pid pid73177
 file pid pid73178
 file pid pid73179
 file pid pid73180
 file pid pid73181
 file pid pid66556
 file pid pid66557
 file pid pid66558
 file pid pid66559
 file pid pid66560
 file pid pid73177
 file pid pid73178
 file pid pid73179
 file pid pid73180
 file pid pid73181
 file pid pid66556
 file pid pid66557
 file pid pid66558
 file pid pid66559
 file pid pid66560
 file pid pid73177
 file pid pid73178
 file pid pid73179
 file pid pid73180
 file pid pid73181
 file pid pid66556
 file pid pid66557
 file pid pid66558
 file pid pid66559
 file pid pid66560
 file pid pid73177
 file pid pid73178
 file pid pid73179
 file pid pid73180
 file pid pid73181
 file pid pid66556
 file pid pid66557
 file pid pi

 file pid pid73178
 file pid pid73179
 file pid pid73180
 file pid pid73181
 file pid pid66556
 file pid pid66557
 file pid pid66558
 file pid pid66559
 file pid pid66560
 file pid pid73177
 file pid pid73178
 file pid pid73179
 file pid pid73180
 file pid pid73181
 file pid pid66556
 file pid pid66557
 file pid pid66558
 file pid pid66559
 file pid pid66560
 file pid pid73177
 file pid pid73178
 file pid pid73179
 file pid pid73180
 file pid pid73181
 file pid pid66556
 file pid pid66557
 file pid pid66558
 file pid pid66559
 file pid pid66560
 file pid pid73177
 file pid pid73178
 file pid pid73179
 file pid pid73180
 file pid pid73181
 file pid pid66556
 file pid pid66557
 file pid pid66558
 file pid pid66559
 file pid pid66560
 file pid pid73177
 file pid pid73178
 file pid pid73179
 file pid pid73180
 file pid pid73181
 file pid pid66556
 file pid pid66557
 file pid pid66558
 file pid pid66559
 file pid pid66560
 file pid pid73177
 file pid pid73178
 file pid pid73179
 file pid pi

 file pid pid73178
 file pid pid73179
 file pid pid73180
 file pid pid73181
 file pid pid66556
 file pid pid66557
 file pid pid66558
 file pid pid66559
 file pid pid66560
 file pid pid73177
 file pid pid73178
 file pid pid73179
 file pid pid73180
 file pid pid73181
 file pid pid66556
 file pid pid66557
 file pid pid66558
 file pid pid66559
 file pid pid66560
 file pid pid73177
 file pid pid73178
 file pid pid73179
 file pid pid73180
 file pid pid73181
 file pid pid66556
 file pid pid66557
 file pid pid66558
 file pid pid66559
 file pid pid66560
 file pid pid73177
 file pid pid73178
 file pid pid73179
 file pid pid73180
 file pid pid73181
 file pid pid66556
 file pid pid66557
 file pid pid66558
 file pid pid66559
 file pid pid66560
 file pid pid73177
 file pid pid73178
 file pid pid73179
 file pid pid73180
 file pid pid73181
 file pid pid66556
 file pid pid66557
 file pid pid66558
 file pid pid66559
 file pid pid66560
 file pid pid73177
 file pid pid73178
 file pid pid73179
 file pid pi

In [7]:
%%time
concatDf = read_and_concatenate_parquets(pidList=pids,incrList=incr)

Z:\FFE\output\scenario0_pid66557.parquet
1
Z:\FFE\output\scenario100_pid66557.parquet
2
Z:\FFE\output\scenario101_pid66557.parquet
3
Z:\FFE\output\scenario102_pid66557.parquet
4
Z:\FFE\output\scenario103_pid66557.parquet
5
Z:\FFE\output\scenario104_pid66557.parquet
6
Z:\FFE\output\scenario105_pid66557.parquet
7
Z:\FFE\output\scenario106_pid66557.parquet
8
Z:\FFE\output\scenario107_pid66557.parquet
9
Z:\FFE\output\scenario108_pid66557.parquet
10
Z:\FFE\output\scenario109_pid66557.parquet
11
Z:\FFE\output\scenario10_pid66557.parquet
12
Z:\FFE\output\scenario110_pid66557.parquet
13
Z:\FFE\output\scenario111_pid66557.parquet
14
Z:\FFE\output\scenario112_pid66557.parquet
15
Z:\FFE\output\scenario113_pid66557.parquet
16
Z:\FFE\output\scenario114_pid66557.parquet
17
Z:\FFE\output\scenario115_pid66557.parquet
18
Z:\FFE\output\scenario116_pid66557.parquet
19
Z:\FFE\output\scenario117_pid66557.parquet
20
Z:\FFE\output\scenario118_pid66557.parquet
21
Z:\FFE\output\scenario119_pid66557.parquet
22


181
Z:\FFE\output\scenario263_pid66557.parquet
182
Z:\FFE\output\scenario264_pid66557.parquet
183
Z:\FFE\output\scenario265_pid66557.parquet
184
Z:\FFE\output\scenario266_pid66557.parquet
185
Z:\FFE\output\scenario267_pid66557.parquet
186
Z:\FFE\output\scenario268_pid66557.parquet
187
Z:\FFE\output\scenario269_pid66557.parquet
188
Z:\FFE\output\scenario26_pid66557.parquet
189
Z:\FFE\output\scenario270_pid66557.parquet
190
Z:\FFE\output\scenario271_pid66557.parquet
191
Z:\FFE\output\scenario272_pid66557.parquet
192
Z:\FFE\output\scenario273_pid66557.parquet
193
Z:\FFE\output\scenario274_pid66557.parquet
194
Z:\FFE\output\scenario275_pid66557.parquet
195
Z:\FFE\output\scenario276_pid66557.parquet
196
Z:\FFE\output\scenario277_pid66557.parquet
197
Z:\FFE\output\scenario278_pid66557.parquet
198
Z:\FFE\output\scenario279_pid66557.parquet
199
Z:\FFE\output\scenario27_pid66557.parquet
200
Z:\FFE\output\scenario280_pid66557.parquet
201
Z:\FFE\output\scenario281_pid66557.parquet
202
Z:\FFE\outp

358
Z:\FFE\output\scenario152_pid66558.parquet
359
Z:\FFE\output\scenario153_pid66558.parquet
360
Z:\FFE\output\scenario154_pid66558.parquet
361
Z:\FFE\output\scenario155_pid66558.parquet
362
Z:\FFE\output\scenario156_pid66558.parquet
363
Z:\FFE\output\scenario157_pid66558.parquet
364
Z:\FFE\output\scenario158_pid66558.parquet
365
Z:\FFE\output\scenario159_pid66558.parquet
366
Z:\FFE\output\scenario15_pid66558.parquet
367
Z:\FFE\output\scenario160_pid66558.parquet
368
Z:\FFE\output\scenario161_pid66558.parquet
369
Z:\FFE\output\scenario162_pid66558.parquet
370
Z:\FFE\output\scenario163_pid66558.parquet
371
Z:\FFE\output\scenario164_pid66558.parquet
372
Z:\FFE\output\scenario165_pid66558.parquet
373
Z:\FFE\output\scenario166_pid66558.parquet
374
Z:\FFE\output\scenario167_pid66558.parquet
375
Z:\FFE\output\scenario168_pid66558.parquet
376
Z:\FFE\output\scenario169_pid66558.parquet
377
Z:\FFE\output\scenario16_pid66558.parquet
378
Z:\FFE\output\scenario170_pid66558.parquet
379
Z:\FFE\outp

535
Z:\FFE\output\scenario41_pid66558.parquet
536
Z:\FFE\output\scenario42_pid66558.parquet
537
Z:\FFE\output\scenario43_pid66558.parquet
538
Z:\FFE\output\scenario44_pid66558.parquet
539
Z:\FFE\output\scenario45_pid66558.parquet
540
Z:\FFE\output\scenario46_pid66558.parquet
541
Z:\FFE\output\scenario47_pid66558.parquet
542
Z:\FFE\output\scenario48_pid66558.parquet
543
Z:\FFE\output\scenario49_pid66558.parquet
544
Z:\FFE\output\scenario4_pid66558.parquet
545
Z:\FFE\output\scenario50_pid66558.parquet
546
Z:\FFE\output\scenario51_pid66558.parquet
547
Z:\FFE\output\scenario52_pid66558.parquet
548
Z:\FFE\output\scenario53_pid66558.parquet
549
Z:\FFE\output\scenario54_pid66558.parquet
550
Z:\FFE\output\scenario55_pid66558.parquet
551
Z:\FFE\output\scenario56_pid66558.parquet
552
Z:\FFE\output\scenario57_pid66558.parquet
553
Z:\FFE\output\scenario58_pid66558.parquet
554
Z:\FFE\output\scenario59_pid66558.parquet
555
Z:\FFE\output\scenario5_pid66558.parquet
556
Z:\FFE\output\scenario60_pid6655

714
Z:\FFE\output\scenario202_pid73177.parquet
715
Z:\FFE\output\scenario203_pid73177.parquet
716
Z:\FFE\output\scenario204_pid73177.parquet
717
Z:\FFE\output\scenario205_pid73177.parquet
718
Z:\FFE\output\scenario206_pid73177.parquet
719
Z:\FFE\output\scenario207_pid73177.parquet
720
Z:\FFE\output\scenario208_pid73177.parquet
721
Z:\FFE\output\scenario209_pid73177.parquet
722
Z:\FFE\output\scenario20_pid73177.parquet
723
Z:\FFE\output\scenario210_pid73177.parquet
724
Z:\FFE\output\scenario211_pid73177.parquet
725
Z:\FFE\output\scenario212_pid73177.parquet
726
Z:\FFE\output\scenario213_pid73177.parquet
727
Z:\FFE\output\scenario214_pid73177.parquet
728
Z:\FFE\output\scenario215_pid73177.parquet
729
Z:\FFE\output\scenario216_pid73177.parquet
730
Z:\FFE\output\scenario217_pid73177.parquet
731
Z:\FFE\output\scenario218_pid73177.parquet
732
Z:\FFE\output\scenario219_pid73177.parquet
733
Z:\FFE\output\scenario21_pid73177.parquet
734
Z:\FFE\output\scenario220_pid73177.parquet
735
Z:\FFE\outp

...

2465
Z:\FFE\output\scenario159_pid73180.parquet
2466
Z:\FFE\output\scenario15_pid73180.parquet
2467
Z:\FFE\output\scenario160_pid73180.parquet
2468
Z:\FFE\output\scenario161_pid73180.parquet
2469
Z:\FFE\output\scenario162_pid73180.parquet
2470
Z:\FFE\output\scenario163_pid73180.parquet
2471
Z:\FFE\output\scenario164_pid73180.parquet
2472
Z:\FFE\output\scenario165_pid73180.parquet
2473
Z:\FFE\output\scenario166_pid73180.parquet
2474
Z:\FFE\output\scenario167_pid73180.parquet
2475
Z:\FFE\output\scenario168_pid73180.parquet
2476
Z:\FFE\output\scenario169_pid73180.parquet
2477
Z:\FFE\output\scenario16_pid73180.parquet
2478
Z:\FFE\output\scenario170_pid73180.parquet
2479
Z:\FFE\output\scenario171_pid73180.parquet
2480
Z:\FFE\output\scenario172_pid73180.parquet
2481
Z:\FFE\output\scenario173_pid73180.parquet
2482
Z:\FFE\output\scenario174_pid73180.parquet
2483
Z:\FFE\output\scenario175_pid73180.parquet
2484
Z:\FFE\output\scenario176_pid73180.parquet
2485
Z:\FFE\output\scenario177_pid73180.pa

2643
Z:\FFE\output\scenario49_pid73180.parquet
2644
Z:\FFE\output\scenario4_pid73180.parquet
2645
Z:\FFE\output\scenario50_pid73180.parquet
2646
Z:\FFE\output\scenario51_pid73180.parquet
2647
Z:\FFE\output\scenario52_pid73180.parquet
2648
Z:\FFE\output\scenario53_pid73180.parquet
2649
Z:\FFE\output\scenario54_pid73180.parquet
2650
Z:\FFE\output\scenario55_pid73180.parquet
2651
Z:\FFE\output\scenario56_pid73180.parquet
2652
Z:\FFE\output\scenario57_pid73180.parquet
2653
Z:\FFE\output\scenario58_pid73180.parquet
2654
Z:\FFE\output\scenario59_pid73180.parquet
2655
Z:\FFE\output\scenario5_pid73180.parquet
2656
Z:\FFE\output\scenario60_pid73180.parquet
2657
Z:\FFE\output\scenario61_pid73180.parquet
2658
Z:\FFE\output\scenario62_pid73180.parquet
2659
Z:\FFE\output\scenario63_pid73180.parquet
2660
Z:\FFE\output\scenario64_pid73180.parquet
2661
Z:\FFE\output\scenario65_pid73180.parquet
2662
Z:\FFE\output\scenario66_pid73180.parquet
2663
Z:\FFE\output\scenario67_pid73180.parquet
2664
Z:\FFE\out

2819
Z:\FFE\output\scenario207_pid66556.parquet
2820
Z:\FFE\output\scenario208_pid66556.parquet
2821
Z:\FFE\output\scenario209_pid66556.parquet
2822
Z:\FFE\output\scenario20_pid66556.parquet
2823
Z:\FFE\output\scenario210_pid66556.parquet
2824
Z:\FFE\output\scenario211_pid66556.parquet
2825
Z:\FFE\output\scenario212_pid66556.parquet
2826
Z:\FFE\output\scenario213_pid66556.parquet
2827
Z:\FFE\output\scenario214_pid66556.parquet
2828
Z:\FFE\output\scenario215_pid66556.parquet
2829
Z:\FFE\output\scenario216_pid66556.parquet
2830
Z:\FFE\output\scenario217_pid66556.parquet
2831
Z:\FFE\output\scenario218_pid66556.parquet
2832
Z:\FFE\output\scenario219_pid66556.parquet
2833
Z:\FFE\output\scenario21_pid66556.parquet
2834
Z:\FFE\output\scenario220_pid66556.parquet
2835
Z:\FFE\output\scenario221_pid66556.parquet
2836
Z:\FFE\output\scenario222_pid66556.parquet
2837
Z:\FFE\output\scenario223_pid66556.parquet
2838
Z:\FFE\output\scenario224_pid66556.parquet
2839
Z:\FFE\output\scenario225_pid66556.pa

2992
Z:\FFE\output\scenario93_pid66556.parquet
2993
Z:\FFE\output\scenario94_pid66556.parquet
2994
Z:\FFE\output\scenario95_pid66556.parquet
2995
Z:\FFE\output\scenario96_pid66556.parquet
2996
Z:\FFE\output\scenario97_pid66556.parquet
2997
Z:\FFE\output\scenario98_pid66556.parquet
2998
Z:\FFE\output\scenario99_pid66556.parquet
2999
Z:\FFE\output\scenario9_pid66556.parquet
3000


MemoryError: 

In [8]:
concatDf.drop_duplicates(['scenario', 'source'])

NameError: name 'concatDf' is not defined

In [ ]:
%%time
countConcatDf = count_fid_occurences(concatDf)

In [ ]:
%%time
mergedDf = Merge(countConcatDf, 'WellWHV_Buildings.shp')
countShape = createShapefile(mergedDf)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# import contextily as ctx
%matplotlib inline
plt.style.use('seaborn-whitegrid')

FinnMeshblockShape = gpd.read_file(pathShapefile / 'Finn_MeshBlockSummary.shp')
countShape = gpd.read_file(pathShapefile / 'Burn3000scenarioWellington.shp')

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(12, 10))

FinnMeshblockShape.plot(ax=ax1, column='WellWHV_Bu', cmap='YlOrRd', alpha=0.7, legend=True)
# ctx.add_basemap(ax1, crs=2193)
countShape.plot(ax=ax2, column='count', cmap='seismic', alpha=0.7, legend=True)
# ctx.add_basemap(ax2, crs=2193)

ax1.set_title('Original burn count at meshblock level')
ax1.ticklabel_format(useOffset=False, style='plain')
ax2.set_title('Network burn count at individual building level')
ax2.ticklabel_format(useOffset=False, style='plain')
ax1.tick_params(direction='out', length=6)
ax2.tick_params(direction="out", length=6)

fig.autofmt_xdate()
plt.tight_layout()
plt.savefig(pathShapefile / 'ComparisonGISvsNetwork_3000Burn.png', dpi=600)
plt.show()